## Data Science Capstone - Milestone Report

## Motivation :
we need to add some line here

 

## Understanding the Problem :
Problem Definition and summary


##  Data Acquisition and Cleaning 
 * Remove Non Ascii Characters : Consider to remove all non ascii characters
 * Case Consideration: 
 No case will Be considered .All words will be converted into lower case.         
 * Remove Stopwords :
 similarly, unlike classification and clustering applications, all words will be included in the model as they represent more than just the primary carriers of the message.
 * Wordform : stemming will not be used as N-Grams are typically based on wordforms (unique, inflected
forms of words). Whereas table and tables are the same lemma, they will be treated as separate words in
this model .
 * Removing punctuations : need to remove All punctuation remards except   " ' " .
 * Removing Numbers: there is no intuition based on the research that numbers will have a great impact on a predication
model and they will be removed
 * Removing Whitespace: this was not discussed directly by Jurafsky and Martin. The intuition is that whitespace has
little to do with context and excess whitespace will be removed
 * Sparse Words: all words will be retained.
 * Typo correction : For one of the complete datasets the function which_misspelled from the ‘qdap’ package was used to determine the most frequent typos/error. Apparently, the most common errors is the omission of the apostroph (e.g. “Im” instead of “I’m” or “theyve” instead of “they’ve”) and the use of ‘internet slang’ abbreviations (e.g. ‘idk’ for “I don’t know”). Also, a lot of links to internet pages are present. Therefore, to allow for some cleaning of the data, these common terms will be replaced (non-exhaustive list shown below).
 * Profanity Filtering : the external link to a textfile with ‘bad words’ was used for profanity filtering (removal of offensive words).
 
     
 
     



In [28]:
# remove non ascii characters 
from unidecode import unidecode
import re
from HTMLParser import HTMLParser
def remove_non_ascii(text):
    return unidecode(unicode(text, encoding = "utf-8"))


# codes from http://stackoverflow.com/a/12824140/735926
RE_EMOJI = re.compile(
        "["
            r"\U00002600-\U000026FF"  # Misc Symbols
            r"\U00002700-\U000027BF"  # Dingbats
            r"\U0001F300-\U0001F5FF"  # Misc Symbols & Pictographs
            r"\U0001F600-\U0001F64F"  # Emoticons
            r"\U0001F680-\U0001F6FF"  # Transport & Map Symbols
        "]"
, re.UNICODE)


# URLs + http:... at the end of truncated tweets
RE_HTTP = re.compile(r"\b(?:https?://\S+|https?:\.\.\.$)")

RE_SPACES = re.compile(r"\s+")
RE_NUMBERS = re.compile("\d+")

RE_HTML_ENTITY = re.compile(r"&[a-z]+;")

RE_RT = re.compile(r"^RT @\S+:?")
RE_END_HASHTAGS = re.compile(r"(?:#\S+\s*)+$")
RE_HASH = re.compile(r"#")
RE_MENTION = re.compile(r"@\S+")

# some tweets contain lots of hashs, e.g.: ####### and this confuse the
# RE_END_HASHTAGS replacement that takes a lot of time. We add this one before
# to remove these repetitions
RE_HASHS = re.compile(r"##+")


SLANGS = [(re.compile(r"\b%s\b" % re.escape(s)),r) for s,r in (
    ("cuz", "because"),
    ("u r", "you are"),
    ("thx", "thanks"),

    # we use legit words instead of Twitter-specific language here
    ("DM", "private message"),
    ("RT", "write"),
    ("tweet", "message"),
)]

DEL_SLANGS = [re.compile(s, re.IGNORECASE) for s in (
    "asap",
    "lol",
    "rofl",
)]


from nltk.corpus import stopwords

def remove_stopwords(words):
    filtered_words = [word for word in words if word not in stopwords.words('english')]
    return filtered_words

def remove_punctuation(words) :
    clean_words = []
    for word in words :
        word =   re.sub('[^A-Za-z\']+', '', word).strip()
        if  word != '': 
            clean_words.append(word)
    return clean_words

words = ['kno04w', "'s", '``', 're\'ason', ".......", '.' ,'?', 'pro~~ceed',":(","https:","asap"]
print remove_punctuation(words)



NAMES_BLACKLIST = set([
    "(", ")", "[", "]", "+", "|", "%", "...", "~", "_", ":[",
    "The", "Of", "In", "", "n", "t", "\n", "m", "x", "N",
    "ll", "re", "ve", "pm",
])

htmlparser = HTMLParser()


def compile_OR_pattern(patterns, *args):
    s = "(?:%s)" % "|".join([re.escape(p) for p in patterns])
    return re.compile(s, *args)


def contains_emoji(text):
    """
    Test if some text contains emojis.
    """
    return RE_EMOJI.search(text) is not None


def unslangize(text):
    """
    Try to remove as many slang words as possible, replacing with their correct
    form.
    """
    for s in DEL_SLANGS:
        text = s.sub("", text)

    for reg, rep in SLANGS:
        text = reg.sub(rep, text)
    return text



def normalize_text(text):
    """
    Return a normalized version of a status, meant to be tokenized by NLTK for
    entity extraction.
    Note that on an Homebrewed Python on OS X the function might print warnings
    like: ::
        RuntimeWarning: Surrogate character u'\\udf05' will be ignored
    You can safely ignore them.
    """
    text = remove_stopwords(text)
    text = remove_punctuation(text)

    return text

print normalize_text(words) 





['know', "'s", "re'ason", 'proceed', 'https', 'asap']
['know', "'s", "re'ason", 'proceed', 'https', 'asap']


In [ ]:
import pandas as pd
import numpy as np
import nltk

import warnings
warnings.filterwarnings("ignore")

blogpath = "./final/en_US/sample.txt"

newspath = "./final/en_US/en_US.news.txt"
twitterpath = "./final/en_US/en_US.twitter.txt"


blog_num_lines = 0
blog_num_words = 0
blog_num_chars = 0
blog_max_line_length = 0

columns = ['File Name','Size ','Documents','Vocabulary (V)','Max line Length' ,'Word Types (T)','TTR (T/V)','Diversity']

words = []

 


with open(twitterpath) as fin:
    for line in fin :
        blog_num_lines += 1
        line = remove_non_ascii(line) # remove non ascii characters
        line = line.lower() #convert into lower case letter
        
        tokens = nltk.word_tokenize(line)
        #print tokens
        tokens = remove_stopwords(tokens) #remove stop words
        #print "after removing stop words:---------"
        #print tokens
        tokens = remove_punctuation(tokens) # remove punctuation and whitespaces and clean numbers
        #print "after removing punctuations:---------"
        #print tokens
        words =  words + tokens

        
        #unique_words.add (line.split())
        blog_num_words += len(words)
        blog_num_chars = len(tokens)
        if blog_max_line_length < blog_num_chars :
            blog_max_line_length = blog_num_chars

#print ( "number of lines in blog file : " , blog_num_lines )
#print ( "number of words in blog file : " , blog_num_words )


unique_words =  set(words)
print len(unique_words)



        
print "Total Unique Words " , len(unique_words) ,"Total words " , len(words)
df_ = pd.DataFrame([['Twitter', 0,blog_num_lines ,blog_num_words ,  blog_max_line_length,0,0,0 ] ],columns=columns)
df_ 



    

## Assume all data are clean


In [3]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"
import pandas as pd
import numpy as np
import nltk
import csv
 
# Read the data and append SENTENCE_START and SENTENCE_END tokens
print "Reading CSV file..."
with open('./final/input.txt', 'rb') as f:
    line = f.readLine()
    
    reader.next()
     
    # Split full comments into sentences
    sentence = nltk.sent_tokenize(reader.decode('utf-8').lower())
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentence]
print "Parsed %d sentences." % (len(sentences))
print sentences[0:20]

Reading CSV file...


AttributeError: '_csv.reader' object has no attribute 'decode'